### Word Embeddings

En esencia, las representaciones de palabras (word embeddings) son representaciones numéricas de datos del mundo real como texto, imágenes, voz o video. Se expresan como vectores de baja dimensionalidad, donde las distancias geométricas de dos vectores en el espacio vectorial son una proyección de las relaciones entre los dos objetos del mundo real que los vectores representan. En otras palabras, te ayudan a proporcionar una representación compacta de datos de diferentes tipos, mientras que simultáneamente te permiten comparar dos objetos de datos diferentes y decir cuán similares o diferentes son en una escala numérica.

Estas representaciones de baja dimensionalidad de datos del mundo real ayudan significativamente a un procesamiento y almacenamiento de datos a gran escala de manera eficiente, actuando como un medio de compresión con pérdida de los datos originales mientras se conservan sus propiedades importantes.

Los "word embeddings" se construyen con diferentes técnica y con modelos de aprendizaje automático, generalmente se pueden usar los word embeddings creados por ChatGPT (OpenAI), Google AI etc.





In [ ]:
!pip install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.21 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu


  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain_community-0.3.9-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_experimental-0.3.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached huggingface_hub-0.23.2-py3-none-any.whl.metadata (12 kB)
  Using cached instructor-1.7.0-py3-none-any.whl.metadata (17 kB)
  Using cached InstructorEmbedding-1.0.1-py2.py3-none-any.whl.metadata (20 kB)
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached torch-2.5.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-

In [4]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
#from dotenv import load_dotenv
from langchain_experimental.text_splitter import SemanticChunker
import os
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR


c:\Users\L01257675\Documents\Agosto-Diciembre2021-DELL\Ago-Diciembre2024\CADI2025\CADI-LLMs\CADI_2025_LLMs\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


### Text embeddings 



In [5]:
embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')
#embeddings = OllamaEmbeddings()


load INSTRUCTOR_Transformer
max_seq_length  512


c:\Users\L01257675\Documents\Agosto-Diciembre2021-DELL\Ago-Diciembre2024\CADI2025\CADI-LLMs\CADI_2025_LLMs\.venv\Lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any i

### Vectorstore

Los embeddings se guardan en databases especiales (aquí se usa FAISS) que permiten almacenar y extraer de manera eficiente la información de los embeddings, comparando  el significado semántico de los queries con el significado semántico de las palabras almacenadas en los embeddings.

### Carga y lee el documento pdf

In [6]:
pdf_path="PeterPan.pdf"

In [7]:
text =''
with open(pdf_path,'rb') as file:
    pdf_reader = PdfReader(file)
    for page in pdf_reader.pages:
        text += page.extract_text()


In [8]:
print(text)

Hace tiempo, allá por 1880, vivía en la ciudad de Londres la familia Darling. Estaba 
formada por el señor y la señora Darling y sus hijos: Wendy, Michael y John. Sin 
olvidarnos de Nana, por supuesto, el perro niñera.  
 
Vivían felices y tranquilos hasta que Peter Pan llegó a sus vidas. Todo comenzó la 
noche en que Nana tenía el día libre y la señora Darling se quedó a cargo de sus hijos. 
Cuando todos, incluida ella, estuvieron dormidos el muchacho entró por la ventana. 
Pero entonces ella se despertó  y se asustó tanto al verle que lanzó un fuerte grito. 
Entonces apareció Nana, que cerró la ventana para evitar que saliera y acabó 
atrapando su sombra. Y así fue como la sombra de Peter Pan acabó en un cajón de la 
casa de los Darling.  
 
Una noche el señor y la señora Darling salieron a cenar a casa de los vecinos del 
número 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en 
quedarse todos dormidos.  
Pero cuando la casa estaba en silencio, entró una diminuta

### Divide el texto 

In [9]:
#text_splitter=SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

In [10]:
docs = text_splitter.split_text(text)
    


In [11]:
type(docs)

list

In [12]:
len(docs)

12

In [13]:
docs[0]

'Hace tiempo, allá por 1880, vivía en la ciudad de Londres la familia Darling. Estaba \nformada por el señor y la señora Darling y sus hijos: Wendy, Michael y John. Sin \nolvidarnos de Nana, por supuesto, el perro niñera.  \n \nVivían felices y tranquilos hasta que Peter Pan llegó a sus vidas. Todo comenzó la \nnoche en que Nana tenía el día libre y la señora Darling se quedó a cargo de sus hijos. \nCuando todos, incluida ella, estuvieron dormidos el muchacho entró por la ventana. \nPero entonces ella se despertó  y se asustó tanto al verle que lanzó un fuerte grito. \nEntonces apareció Nana, que cerró la ventana para evitar que saliera y acabó \natrapando su sombra. Y así fue como la sombra de Peter Pan acabó en un cajón de la \ncasa de los Darling.  \n \nUna noche el señor y la señora Darling salieron a cenar a casa de los vecinos del \nnúmero 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en \nquedarse todos dormidos.'

In [14]:
docs[1]

'número 27. Los niños se quedaron en casa al cuidado de Nana y no tardaron en \nquedarse todos dormidos.  \nPero cuando la casa estaba en silencio, entró una diminuta hada revoloteando a gran \nvelocidad y tras ella, Peter Pan, dispuesto a recuperar su sombra. La encontró en el \ncajón en el que la había guardado Nana pero se entristeció mucho cuando comprobó \nque la sombra no le seguía. Probó a pegársela con jabón pero no dio resultado y \ndesesperado se se ntó en el suelo a llorar.  \n \n- ¿Quién está llorando? - preguntó Wendy, a quien despertaron los sollozos.  \n- Soy yo - contestó Peter  \n- ¿Cómo te llamas? - preguntó la niña, aunque ella estaba casi segura de saber quien \nera \n- Peter Pan  \n- ¿Y qué te pasa Peter?  \n- Que no consigo que mi sombra se me quede pegada  \n- Tranquilo. Creo que podré cosértela  \n \nWendy ayudó a Peter y mientras los dos niños comenzaron a hacerse amigos.  \n \n- Yo vivo en el País de Nunca Jamás. Es maravilloso, allí eres siempre un niño y no'

### Crea el vectorstore

In [16]:
vectorstore = FAISS.from_texts(docs, embeddings)


### Guardar el vectorstore


In [ ]:
vectorstore.save_local("faiss_index")

### Cargar el vectorstore

In [ ]:
vectorstore = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)
